
# NetCDF to dfs2

## Weather forcing

The first example uses data from ECMWF ERA5 obtained from [CDS](https://cds.climate.copernicus.eu/#!/home)

A common format for multidimensional data is [NetCDF](https://www.unidata.ucar.edu/software/netcdf/).

An excellent python package for working with NetCDF in Python is [Xarray](http://xarray.pydata.org/en/stable/).

To work with NetCDF data first we need to install the `netcdf4` and `xarray` packages!  

Opening a NetCDF file with Xarray returns an `xarray.Dataset`

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import mikeio

ds = xr.open_dataset("data/era5_small.nc")
ds

One of the datam variables is the t2m (Air temperature at 2m above the ground) which we can access in two different ways:

1. Using the string "t2m"
2. Or as a property of the Dataset object `.t2m`

In [ ]:
ds["t2m"]

In [ ]:
ds.t2m

To select the first timestep, we can use the `xarray.Dataset.isel` method.

In [ ]:
ds0 = ds.isel(time=0)
ds0

The dataset from the first timestep is also a Dataset, except it no longer has a time dimension.

To plot a map of the air temperature we can use select the `t2m` property and call the plot method.

In [ ]:
ds0.t2m.plot()

## Pressure

Atmospheric pressure comes in different flavors.

`Surface pressure` (sp) is the pressure at ground level, i.e. following the terrain.

In the following plot you can clearly see the decrease in surface pressure with altitude in areas with mountains.

This is *NOT* the variable used by MIKE 21 FM HD.

In [ ]:
ds0.sp.plot()

MIKE 21 FM HD expects the atmospheric pressure reduced to Mean Sea Level. (`msl`) 

In the following plot the pressure field has a much smaller range, since the variation with altitude have been removed.

In [ ]:
ds0.msl.plot()

## Wind

The wind velocity components U,V at 10m above the ground, are found in the two data variables, `u10` and `v10`

In [ ]:
ds0.u10.long_name

In [ ]:
ds0.v10.long_name

## Convert to dfs2

### Sub-region

In this case the NetCDF file covers a much larger area, than is required for our modelling work, so first step is to create a spatial subset to match our Area of Interest.

Since the latitude dimension is ordered in decreasing order in this dataset, the slice has to go from high to low.

In [ ]:
ds_aoi = ds.sel(longitude=slice(10,30), 
                latitude=slice(60,45)) # N -> S
ds_aoi

Make a plot to check that the area is the one we expect.

In [ ]:
ds_aoi.isel(time=0).u10.plot()

### Time

Now we are ready to convert this data to dfs2.

First step is to create a time axis, which can be understood by MIKE IO.

In [ ]:
time = pd.DatetimeIndex(ds.time)
time

Then create the spatial axis

In [ ]:
g = mikeio.Grid2D(x=ds_aoi.longitude, y=ds_aoi.latitude[::-1], projection="LONG/LAT")
g

The next step is to collect the variables that we need, and at the same time do unit conversion where it is necessary. In this case for pressure and temperature.

In [ ]:
das = [
    mikeio.DataArray(np.flip(ds_aoi.msl.values, axis=1) / 100.0, time=time, geometry=g, item=ItemInfo("MSLP", EUMType.Pressure, EUMUnit.hectopascal)),
    mikeio.DataArray(np.flip(ds_aoi.u10.values, axis=1), time=time, geometry=g, item=ItemInfo("U 10m", EUMType.Wind_Velocity, EUMUnit.meter_per_sec)),
    mikeio.DataArray(np.flip(ds_aoi.v10.values, axis=1), time=time, geometry=g, item=ItemInfo("V 10m", EUMType.Wind_Velocity, EUMUnit.meter_per_sec)),
    mikeio.DataArray(np.flip(ds_aoi.t2m.values, axis=1) - 273.15, time=time, geometry=g, item=ItemInfo("Temperature", EUMType.Temperature, EUMUnit.degree_Celsius))
]

In [ ]:
my_ds = mikeio.Dataset(das)
my_ds

Let's plot to confirm that it looks alright

In [ ]:
my_ds.U_10m.plot(cmap="coolwarm",vmin=-8,vmax=8);

The dimensions for at  DFS2 are expected to be (t, y, x), which matches the ones used by this dataset.

This convention is recommended by the [CF convention](http://cfconventions.org/cf-conventions/cf-conventions.html#dimensions)

In [ ]:
ds.Conventions

In [ ]:
ds.u10.dims

In [ ]:
my_ds.dims

The final step is to write the dataset to a dfs2 file. The Dataset already contains all the information needed.

In [ ]:
my_ds.to_dfs("era5_aoi.dfs2")


Screenshot of U 10m from MIKE Zero

![](images/era5_u10_dfs2.png)